In [1]:
import pandas as pd

# Load the dataset
file_path = 'moodtaggiven.csv'
data = pd.read_csv(file_path)

# Drop the existing Mood column
if 'Mood' in data.columns:
    data = data.drop(columns=['Mood'])

# Define a function for mood allocation based on research-backed logic
def assign_mood(recipe_name, recipe_ingredients, recipe_description):
    # Combine relevant text fields
    combined_text = f"{recipe_name} {recipe_ingredients} {recipe_description}".lower()

    # Energizing: Spicy foods, caffeine, and ingredients boosting alertness
    if any(keyword in combined_text for keyword in ['spicy', 'caffeine', 'coffee', 'chili', 'pepper', 'turmeric']):
        return 'energizing'
    
    # Calming: Warm, comforting foods like soups or herbal ingredients
    elif any(keyword in combined_text for keyword in ['comfort', 'warm', 'soup', 'broth', 'oats', 'slow-cooked', 'chamomile']):
        return 'calming'
    
    # Happy: Sweet foods, chocolate, and fruits known for pleasure-inducing properties
    elif any(keyword in combined_text for keyword in ['chocolate', 'sweet', 'sugar', 'fruit', 'honey', 'dessert']):
        return 'happy'
    
    # Relaxing: Herbal and soothing ingredients promoting calmness
    elif any(keyword in combined_text for keyword in ['herbal', 'lavender', 'chamomile', 'calm', 'soothing', 'mint']):
        return 'relaxing'
    
    # Default mood if no keywords match
    return 'neutral'

# Apply the mood assignment function to each row
data['Mood_Tag'] = data.apply(
    lambda row: assign_mood(
        recipe_name=row.get('Dish', ''),
        recipe_ingredients=row.get('Recipe Ingredients', ''),
        recipe_description=row.get('Recipe', '')
    ), 
    axis=1
)

# Save the updated dataset
updated_file_path = 'Updated_Recipes_With_Moods.csv'
data.to_csv(updated_file_path, index=False)
print(f"Updated dataset saved to {updated_file_path}")


Updated dataset saved to Updated_Recipes_With_Moods.csv


In [5]:
import pandas as pd

# Load both datasets
food_data_final_path = 'food_data_final.csv'
updated_recipes_with_moods_path = 'Updated_Recipes_With_Moods.csv'

# Load the data
food_data_final = pd.read_csv(food_data_final_path)
updated_recipes_with_moods = pd.read_csv(updated_recipes_with_moods_path)

# Rename columns to match the desired structure
food_data_final.rename(columns={
    'Dish': 'name',
    'Recipe Link': 'recipe_link',
    'Recipe Ingredients': 'ingredients',
    'Recipe': 'recipe',
    'Image Link': 'image_link'
}, inplace=True)

updated_recipes_with_moods.rename(columns={
    'Dish': 'name',
    'Recipe Link': 'recipe_link',
    'Recipe Ingredients': 'ingredients',
    'Recipe': 'recipe',
    'Image Link': 'image_link',
    'Mood_Tag': 'mood_tag'
}, inplace=True)

# Add recipe_id to both datasets
food_data_final.insert(0, 'recipe_id', range(1, len(food_data_final) + 1))
updated_recipes_with_moods.insert(0, 'recipe_id', range(len(food_data_final) + 1, len(food_data_final) + len(updated_recipes_with_moods) + 1))

# Remove duplicate columns while concatenating
columns_to_keep = [
    'recipe_id', 'name', 'recipe_link', 'ingredients', 'mood_tag', 
    'image_link', 'recipe'
]

# Ensure both datasets have these columns
food_data_final = food_data_final[columns_to_keep]
updated_recipes_with_moods = updated_recipes_with_moods[columns_to_keep]

# Combine the datasets
combined_data = pd.concat([food_data_final, updated_recipes_with_moods], ignore_index=True)

# Function to assign diet_type based on ingredients
def assign_diet_type(ingredients):
    if pd.isnull(ingredients):
        return 'other'
    ingredients_list = ingredients.lower()
    if any(i in ingredients_list for i in ['meat', 'chicken', 'beef', 'pork']):
        return 'omnivore'
    elif any(i in ingredients_list for i in ['fish', 'salmon', 'tuna']):
        return 'pescatarian'
    elif any(i in ingredients_list for i in ['egg', 'dairy', 'milk']):
        return 'vegetarian'
    elif 'cheese' in ingredients_list:
        return 'vegetarian'
    elif any(i in ingredients_list for i in ['tofu', 'lentil']):
        return 'vegan'
    else:
        return 'other'

# Apply the diet_type function
combined_data['diet_type'] = combined_data['ingredients'].apply(assign_diet_type)

# Save the cleaned and combined dataset
output_path = '/mnt/data/combined_recipes_with_diet_and_moods.csv'
combined_data.to_csv(output_path, index=False)

output_path


KeyError: "['mood_tag'] not in index"